In [14]:
import pandas as pd
import numpy as np

In [15]:
# read in Players_Complete csv file
total_player_df = pd.read_csv('players_complete.csv')

In [16]:
# Change Column Header from SN to Screen Name
total_player_df = total_player_df.rename(columns={'SN':'Screen Name'})
total_player_df.head()

,Player ID,Screen Name,Age,Gender
0,0,Marughi89,21,Male
1,1,Lirtedy26,40,Male
2,2,Chamistast30,7,Male
3,3,Lisirra25,24,Male
4,4,Lirtim36,23,Male


In [17]:
# Count the total number of unique players
player_count = len(total_player_df['Screen Name'].unique())
player_count

1161